In [1]:
# 中央通訊社
# 爬蟲套件
import requests
from bs4 import BeautifulSoup
import json
from fake_useragent import UserAgent
# 資料整理類套件
import pandas as pd
import re
import time
import random
import datetime
from tomorrow import threads
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pymongo import MongoClient
from datetime import datetime
import pymongo

options = Options() # 啟動無頭模式
options.add_argument('--headless')  #規避google bug
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
# CHROMEDRIVER_PATH = "/Users/ryanwu/chromedriver"
CHROMEDRIVER_PATH = "/root/chromedriver"
driver = webdriver.Chrome(CHROMEDRIVER_PATH,options=options)

In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [3]:
def GetAllLinks():
    links = []
    driver.get('https://www.cna.com.tw/list/aipl.aspx')
    while True:
        try:
            driver.find_element_by_link_text("看更多內容").click()
            time.sleep(2)
        except:
            break

    soup = BeautifulSoup(driver.page_source)
    for i in soup.find("ul",id="myMainList").find_all("li"):
        if i.find("a").get("href")[:22] == "https://www.cna.com.tw":
            links.append(i.find("a").get("href"))
    driver.close()
    return links

In [4]:
def GetCNA(response):
    soup = BeautifulSoup(response.text)
    Url = soup.find("link").get("href")
    Title = soup.find("h1").text
    CreateTime = soup.find("div",class_="updatetime").find("span").text
    CreateTime = datetime.strptime(CreateTime,'%Y/%m/%d %H:%M')
    Content_list = []
    for i in soup.find("div",class_="paragraph").find_all("p"):
        Content_list.append(i.text)
    Content = "".join(Content_list)
    Tag = soup.find("meta",{"name":"keywords"}).get("content").split(",")
    
    ndf = pd.DataFrame(data = [{'Source':'News',         
                                'Site':'中央通訊社',
                                'Url':Url,
                                'Author':'無作者',
                                'Title':Title,
                                'CreateTime':CreateTime,
                                'Content':Content,
                                'Tag':Tag}],
                       columns = ['Source', 'Site', 'Url', 'CreateTime', 'Author', 'Title', 'Content','Tag']) 
    print(Url)
    return ndf



In [5]:
#測試抓取一篇文章並存為是否成功
# testurl = 'https://www.cna.com.tw/news/firstnews/201911260346.aspx'   #自定義
# resp = requests.get(testurl,headers=headers)
# GetCNA(resp)

In [6]:
# 多線程爬蟲定義
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [7]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = GetCNA(response)
            allData.append(ndf)
        except:
            pass

    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    
    return df

In [8]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")


In [ ]:
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

In [9]:
links = GetAllLinks()
df = CrawlAllNews(links)
dataframe_to_mongo_test(df)

There are 100 links in pages.
There are 100 News in DataFrame.


ServerSelectionTimeoutError: 192.168.1.126:27017: [Errno 64] Host is down